In [1]:
from sql_util.dbinfo import get_all_db_info_path
from pprint import pprint

In [15]:
import process_sql

import pickle

dataset = None

with open("dataset/classical_test.pkl", "rb") as file:
    dataset = pickle.load(file)


In [16]:
import random

random.seed(10)

random.shuffle(dataset)

train_data = dataset[:int(len(dataset) * .8)]
test_data = dataset[int(len(dataset) * .8):]

In [17]:
inputs = [ex['text'] for ex in train_data]
targets = [ex['query'] for ex in train_data]
target_db_paths = [ex['db_path'] for ex in train_data]

In [18]:
targets[10]

'SELECT COUNT( DISTINCT ( PUBLICATIONalias0.TITLE ) ) FROM AUTHOR AS AUTHORalias0 , CONFERENCE AS CONFERENCEalias0 , PUBLICATION AS PUBLICATIONalias0 , WRITES AS WRITESalias0 WHERE AUTHORalias0.NAME = "H. V. Jagadish" AND CONFERENCEalias0.NAME = "VLDB" AND PUBLICATIONalias0.CID = CONFERENCEalias0.CID AND WRITESalias0.AID = AUTHORalias0.AID AND WRITESalias0.PID = PUBLICATIONalias0.PID ;'

In [19]:
from process_sql import get_sql, get_schema, Schema
import os

num = 10

schema = Schema(get_schema(target_db_paths[num]))
sql = get_sql(schema, targets[num])

sql

AssertionError: Error col: (

In [9]:
original_database_path = 'concert_singer.sqlite'
table_column2properties, column_references, table_column2elements = get_all_db_info_path(original_database_path)

In [10]:
for k, v in table_column2elements.items():
    if k[0] == 'stadium':
        print(k, v)

('stadium', 'STADIUM_ID') [1, 2, 3, 4, 5, 6, 7, 9, 10]
('stadium', 'LOCATION') ['Raith Rovers', 'Ayr United', 'East Fife', "Queen's Park", 'Stirling Albion', 'Arbroath', 'Alloa Athletic', 'Peterhead', 'Brechin City']
('stadium', 'NAME') ["Stark's Park", 'Somerset Park', 'Bayview Stadium', 'Hampden Park', 'Forthbank Stadium', 'Gayfield Park', 'Recreation Park', 'Balmoor', 'Glebe Park']
('stadium', 'CAPACITY') [10104, 11998, 2000, 52500, 3808, 4125, 3100, 4000, 3960]
('stadium', 'HIGHEST') [4812, 2363, 1980, 1763, 1125, 921, 1057, 837, 780]
('stadium', 'LOWEST') [1294, 1057, 533, 466, 404, 411, 331, 400, 315]
('stadium', 'AVERAGE') [2106, 1477, 864, 730, 642, 638, 637, 615, 552]


In [11]:
gold_query = 'select location from stadium where capacity < 100000 ;'

In [12]:
from fuzz.neighbor import generate_neighbor_queries_path

neighbors = generate_neighbor_queries_path(original_database_path, gold_query)
import random
random.shuffle(neighbors)
pprint(neighbors[:10])
print('There are', len(neighbors), 'neighbor queries generated for the gold query.')

['select location from stadium where capacity < -13376 ;',
 'select location from stadium where capacity < -63644 ;',
 'select location from stadium where capacity < 100001 ;',
 'select location from stadium where capacity < -8615 ;',
 'select location from stadium where LOWEST < 100000 ;',
 'select location from stadium where capacity < -96720 ;',
 'select location from stadium where 100000 ;',
 'select location from stadium where STADIUM_ID < 100000 ;',
 'select location from stadium where capacity != 100000 ;',
 'select location from stadium where LOCATION < 100000 ;']
There are 40 neighbor queries generated for the gold query.


In [13]:
from fuzz.fuzz import generate_random_db_with_queries_wrapper

generate_random_db_with_queries_wrapper((original_database_path, original_database_path, [gold_query], {}))

now fuzzing based on database concert_singer.sqlite, target path concert_singer.sqlite.


In [14]:
from utils import data_parser